# 210928 TIL

## 오늘 배운 것

### selenium
#### 웹 어플리케이션 테스트를 위한 프레임워크
#### 다양한 브라우저 작동을 지원하며 크롤링에 활용 가능

### 브라우저 컨트롤
- `driver = webdriver.Chrome('./chromedriver.exe')` 크롬창 열기 

- `driver.get(url)` 설정한 url로 데이터 get함수로 요청, 해당 사이트로 이동

- `driver.find_element_by_xpath('xpath value').send_keys(key_word)` xpath값으로 해당 경로에 값 입력

- `driver.find_element_by_xpath('xpath value').click()` xpath값에 해당하는 버튼 클릭

### 인스타그램 게시글 날짜, 좋아요, 해시태그 크롤링

In [ ]:
# 패키지 로드
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [ ]:
# 크롤링 데이터를 저장할 리스트 생성
datetime = []
like = []
tag = []

# 크롬드라이버 실행 후 최대화

driver = webdriver.Chrome('./chromedriver.exe')
driver.implicitly_wait(10)
driver.maximize_window()
time.sleep(1)


# 인스타그램 페이지로 이동

url = 'https://www.instagram.com/?hl=ko'
driver.get(url)
driver.implicitly_wait(10)

# 로그인

iid = input('아이디, 또는 이메일을 입력하세요')
ipw = input('비밀번호를 입력하세요')

driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys(iid)
time.sleep(0.5)

driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys(ipw)
time.sleep(0.5)

driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button/div').click()
driver.implicitly_wait(10)
time.sleep(0.5)

# 알람 넘어가기

driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/div/button').click()
driver.implicitly_wait(10)
time.sleep(0.5)

try:
    driver.find_element_by_xpath('/html/body/div[5]/div/div/div/div[3]/button[2]').click()
except NoSuchElementException:
    driver.find_element_by_xpath('/html/body/div[6]/div/div/div/div[3]/button[2]').click()

# 키워드로 검색
keyword = input('검색할 키워드를 입력하세요: ')
driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input').send_keys(f'#{keyword}')
time.sleep(3)
driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[3]/div/div[2]/div/div[1]/a/div').click()
time.sleep(5)


# 해당 키워드로 검색한 50개 포스팅의 작성일자, 좋아요 수, 태그 크롤링 하기
driver.find_element_by_xpath('//*[@id="react-root"]/section/main/article/div[1]/div/div/div[1]/div[1]/a/div[1]').click()
time.sleep(1)
print('1번째 게시물 크롤링')
html = BeautifulSoup(driver.page_source, 'html.parser')

datetime.append(html.select('time')[0]['title']) # 시간

like.append(html.select('a.zV_Nj')[0].text) # 좋아요

hashtag = []
for i in html.select('a.xil3i'): # 태그
    hashtag.append(i.text)
tag.append(hashtag)

driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/a[1]').click()
time.sleep(3)

# 나머지 반복
for i in range(49):
    print(f'{i+2}번째 게시물 크롤링')
    html = BeautifulSoup(driver.page_source, 'html.parser')
    hashtag = []
    datetime.append(html.select('time')[0]['title'])

    try:
        like.append(html.select('a.zV_Nj')[0].text)
    except IndexError:
        like.append('좋아요 0개')

    for h in html.select('a.xil3i'):
        hashtag.append(h.text)
    tag.append(hashtag)
    driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/a[2]').click()
    time.sleep(4)


In [ ]:
# 데이터프레임 생성
df = pd.DataFrame({'date' : datetime,
                 'like' : like,
                  'hashtag' : tag})

df

In [ ]:
# csv로 내보내기
df.to_csv("title.csv", index=False, encoding="utf-8-sig")